In [2]:
import pandas as pd

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv("short_term_data/turnstile_170121.txt")

In [5]:
#FIX COLUMNS NAME
df.columns=[c.strip() for c in  df.columns]

In [6]:
df=df.drop(["DESC","DIVISION"],axis=1)

In [7]:
#SORT LINENAME
df["LINENAME"]=df["LINENAME"].apply(lambda x: ''.join(sorted(x)))

In [8]:
#CREATE TIMe
df['T_H']=pd.to_datetime(df['DATE'] + ' ' + df['TIME'], format='%m/%d/%Y %H:%M:%S')

In [9]:
df['T_H_previous'] = df.groupby(['C/A', 'UNIT',  'STATION', 'LINENAME','SCP'])['T_H'].shift(1)

In [10]:
#TOT FLOW
df["TOT"]=df.apply(lambda row: row["ENTRIES"]+row["EXITS"],axis=1)

In [11]:
#create previous TOT
df['TOT_previous'] = df.groupby(['C/A', 'UNIT',  'STATION', 'LINENAME','SCP'])['TOT'].shift(1)
#creates NaN at the begining of the group!

In [12]:
df=df.dropna(subset = ['TOT_previous']) 

In [13]:
df["weekday"]=df['T_H'].dt.dayofweek #

In [14]:
df=df.sort_values(["STATION", "LINENAME", "C/A","SCP",'T_H'])
df["Delta_people"]=df.apply(lambda row: row["TOT"]-row["TOT_previous"],axis=1)

df=df[df["Delta_people"]>0]

In [15]:
df["Delta_time"]=df.apply(lambda row: (row["T_H"]-row["T_H_previous"]).seconds,axis=1)

In [20]:
resampled=df.set_index('T_H').groupby(['C/A', 'UNIT', 'STATION', 'LINENAME',"weekday"]).resample("4H").sum()

ValueError: cannot insert weekday, already exists

In [36]:
resampled=resampled[["TOT_previous","Delta_people","Delta_time"]]

In [37]:
resampled["hr"]=resampled.reset_index().set_index("T_H").index.hour

In [38]:
resampled

TOT_previous  \
C/A   UNIT STATION       LINENAME weekday T_H                                 
A002  R051 59 ST         456NQRW  0       2017-01-16 00:00:00    78224828.0   
                                          2017-01-16 04:00:00    78225385.0   
                                          2017-01-16 08:00:00    78225979.0   
                                          2017-01-16 12:00:00    78229884.0   
                                          2017-01-16 16:00:00    78233430.0   
                                          2017-01-16 20:00:00    78239507.0   
                                  1       2017-01-17 00:00:00    78242225.0   
                                          2017-01-17 04:00:00    78242675.0   
                                          2017-01-17 08:00:00    78244985.0   
                                          2017-01-17 12:00:00    78250588.0   
                                          2017-01-17 16:00:00    78253986.0   
                                          2017-01-17 20:00:00    78262308.0   
                                  2       2017-01-18 00:00:00    78265963.0   
                                          2017-01-18 04:00:00    78266508.0   
                                          2017-01-18 08:00:00    78267493.0   
                                          2017-01-18 12:00:00    78274435.0   
                                          2017-01-18 16:00:00    78277817.0   
                                          2017-01-18 20:00:00    78286595.0   
                                  3       2017-01-19 00:00:00    78290606.0   
                                          2017-01-19 04:00:00    78291273.0   
                                          2017-01-19 08:00:00    78292231.0   
                                          2017-01-19 12:00:00    78299121.0   
                                          2017-01-19 16:00:00    78304034.0   
                                          2017-01-19 20:00:00    78311495.0   
                                  4       2017-01-20 00:00:00    78315461.0   
                                          2017-01-20 04:00:00    78316157.0   
                                          2017-01-20 08:00:00    78317026.0   
                                          2017-01-20 12:00:00    78323429.0   
                                          2017-01-20 16:00:00    78326801.0   
                                          2017-01-20 20:00:00    78335499.0   
...                                                                     ...   
TRAM2 R469 RIT-ROOSEVELT R        1       2017-01-17 20:00:00     9117316.0   
                                  2       2017-01-18 00:00:00     9117622.0   
                                          2017-01-18 04:00:00     6026509.0   
                                          2017-01-18 08:00:00    20297452.0   
                                          2017-01-18 12:00:00     9119213.0   
                                          2017-01-18 16:00:00     9121911.0   
                                          2017-01-18 20:00:00     9119800.0   
                                  3       2017-01-19 00:00:00     9120206.0   
                                          2017-01-19 04:00:00    18240716.0   
                                          2017-01-19 08:00:00     9120394.0   
                                          2017-01-19 12:00:00     9123353.0   
                                          2017-01-19 16:00:00     9122118.0   
                                          2017-01-19 20:00:00     9122662.0   
                                  4       2017-01-20 00:00:00     9123132.0   
                                          2017-01-20 04:00:00     6031772.0   
                                          2017-01-20 08:00:00     9123297.0   
                                          2017-01-20 12:00:00     9126228.0   
                                          2017-01-20 16:00:00     9127441.0   
                                  

In [320]:
#Filetring hours
df=df.set_index('T_H')
start_hour='06:59'
end_hour='19:01'
df=df.between_time(start_hour,end_hour)
df=df.reset_index()

,T_H,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,T_H_previous,TOT,TOT_previous,Delta_people,Delta_time
17795,2017-01-14 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,07:00:00,REGULAR,11939408,13103732,2017-01-14 03:00:00,25043140,2.504291e+07,228.0,14400
17796,2017-01-14 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,11:00:00,REGULAR,11939672,13104331,2017-01-14 07:00:00,25044003,2.504314e+07,863.0,14400
17797,2017-01-14 15:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,15:00:00,REGULAR,11940179,13105231,2017-01-14 11:00:00,25045410,2.504400e+07,1407.0,14400
17798,2017-01-14 19:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,19:00:00,REGULAR,11940675,13106121,2017-01-14 15:00:00,25046796,2.504541e+07,1386.0,14400
17799,2017-01-15 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,07:00:00,REGULAR,11941077,13107432,2017-01-15 03:00:00,25048509,2.504835e+07,163.0,14400
17800,2017-01-15 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,11:00:00,REGULAR,11941214,13107909,2017-01-15 07:00:00,25049123,2.504851e+07,614.0,14400
17801,2017-01-15 15:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,15:00:00,REGULAR,11941542,13108748,2017-01-15 11:00:00,25050290,2.504912e+07,1167.0,14400
17802,2017-01-15 19:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,19:00:00,REGULAR,11941891,13109618,2017-01-15 15:00:00,25051509,2.505029e+07,1219.0,14400
17803,2017-01-16 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/16/2017,07:00:00,REGULAR,11942194,13110658,2017-01-16 03:00:00,25052852,2.505257e+07,283.0,14400
17804,2017-01-16 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/16/2017,11:00:00,REGULAR,11942717,13111472,2017-01-16 07:00:00,25054189,2.505285e+07,1337.0,14400


In [321]:
df.drop(['DESC'], axis=1)

,T_H,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,T_H_previous,TOT,TOT_previous,Delta_people,Delta_time
17795,2017-01-14 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,07:00:00,11939408,13103732,2017-01-14 03:00:00,25043140,2.504291e+07,228.0,14400
17796,2017-01-14 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,11:00:00,11939672,13104331,2017-01-14 07:00:00,25044003,2.504314e+07,863.0,14400
17797,2017-01-14 15:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,15:00:00,11940179,13105231,2017-01-14 11:00:00,25045410,2.504400e+07,1407.0,14400
17798,2017-01-14 19:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,19:00:00,11940675,13106121,2017-01-14 15:00:00,25046796,2.504541e+07,1386.0,14400
17799,2017-01-15 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,07:00:00,11941077,13107432,2017-01-15 03:00:00,25048509,2.504835e+07,163.0,14400
17800,2017-01-15 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,11:00:00,11941214,13107909,2017-01-15 07:00:00,25049123,2.504851e+07,614.0,14400
17801,2017-01-15 15:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,15:00:00,11941542,13108748,2017-01-15 11:00:00,25050290,2.504912e+07,1167.0,14400
17802,2017-01-15 19:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,19:00:00,11941891,13109618,2017-01-15 15:00:00,25051509,2.505029e+07,1219.0,14400
17803,2017-01-16 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/16/2017,07:00:00,11942194,13110658,2017-01-16 03:00:00,25052852,2.505257e+07,283.0,14400
17804,2017-01-16 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/16/2017,11:00:00,11942717,13111472,2017-01-16 07:00:00,25054189,2.505285e+07,1337.0,14400


In [322]:
df

,T_H,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,T_H_previous,TOT,TOT_previous,Delta_people,Delta_time
17795,2017-01-14 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,07:00:00,REGULAR,11939408,13103732,2017-01-14 03:00:00,25043140,2.504291e+07,228.0,14400
17796,2017-01-14 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,11:00:00,REGULAR,11939672,13104331,2017-01-14 07:00:00,25044003,2.504314e+07,863.0,14400
17797,2017-01-14 15:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,15:00:00,REGULAR,11940179,13105231,2017-01-14 11:00:00,25045410,2.504400e+07,1407.0,14400
17798,2017-01-14 19:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/14/2017,19:00:00,REGULAR,11940675,13106121,2017-01-14 15:00:00,25046796,2.504541e+07,1386.0,14400
17799,2017-01-15 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,07:00:00,REGULAR,11941077,13107432,2017-01-15 03:00:00,25048509,2.504835e+07,163.0,14400
17800,2017-01-15 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,11:00:00,REGULAR,11941214,13107909,2017-01-15 07:00:00,25049123,2.504851e+07,614.0,14400
17801,2017-01-15 15:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,15:00:00,REGULAR,11941542,13108748,2017-01-15 11:00:00,25050290,2.504912e+07,1167.0,14400
17802,2017-01-15 19:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/15/2017,19:00:00,REGULAR,11941891,13109618,2017-01-15 15:00:00,25051509,2.505029e+07,1219.0,14400
17803,2017-01-16 07:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/16/2017,07:00:00,REGULAR,11942194,13110658,2017-01-16 03:00:00,25052852,2.505257e+07,283.0,14400
17804,2017-01-16 11:00:00,H007,R248,00-00-00,1 AV,L,BMT,01/16/2017,11:00:00,REGULAR,11942717,13111472,2017-01-16 07:00:00,25054189,2.505285e+07,1337.0,14400


In [82]:
pd.set_option("display.max_rows",501)
A.head(500)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
T_H,,,,,,,,,,,
2017-01-14 09:00:00,A006,R079,00-00-00,5 AV/59 ST,NQRW,BMT,01/14/2017,09:00:00,REGULAR,2648060,4551108
2017-01-15 09:00:00,A006,R079,00-00-00,5 AV/59 ST,NQRW,BMT,01/15/2017,09:00:00,REGULAR,2649090,4552518
2017-01-16 08:26:08,A006,R079,00-00-00,5 AV/59 ST,NQRW,BMT,01/16/2017,08:26:08,REGULAR,2649909,4554015
2017-01-16 09:00:00,A006,R079,00-00-00,5 AV/59 ST,NQRW,BMT,01/16/2017,09:00:00,REGULAR,2649916,4554132
2017-01-17 09:00:00,A006,R079,00-00-00,5 AV/59 ST,NQRW,BMT,01/17/2017,09:00:00,REGULAR,2651167,4556287
2017-01-18 09:00:00,A006,R079,00-00-00,5 AV/59 ST,NQRW,BMT,01/18/2017,09:00:00,REGULAR,2652943,4559111
2017-01-19 09:00:00,A006,R079,00-00-00,5 AV/59 ST,NQRW,BMT,01/19/2017,09:00:00,REGULAR,2654752,4562074
2017-01-14 09:00:00,A006,R079,00-00-01,5 AV/59 ST,NQRW,BMT,01/14/2017,09:00:00,REGULAR,3887362,2469261
2017-01-15 09:00:00,A006,R079,00-00-01,5 AV/59 ST,NQRW,BMT,01/15/2017,09:00:00,REGULAR,3888284,2469718


In [61]:
start_date='2017-01-14'
end_date='2017-07-14'

date_list = [my_dt_ob.year, my_dt_ob.month, my_dt_ob.day, my_dt_ob.hour, my_dt_ob.minute, my_dt_ob.second]

DatetimeIndex(['2017-01-14', '2017-01-15', '2017-01-16', '2017-01-17',
               '2017-01-18', '2017-01-19', '2017-01-20', '2017-01-21',
               '2017-01-22', '2017-01-23',
               ...
               '2017-07-05', '2017-07-06', '2017-07-07', '2017-07-08',
               '2017-07-09', '2017-07-10', '2017-07-11', '2017-07-12',
               '2017-07-13', '2017-07-14'],
              dtype='datetime64[ns]', length=182, freq='D')

In [ ]:
start_date='2017-01-14 03:00:00'
end_date='2017-01-21'
time_series=pd.date_range(start_date, end_date, freq='4h')
time_series

df[~df["T_H"].isin(time_series)]df.groupby([['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME'], pd.Grouper(key='T_H', freq='4H')])

In [ ]:
df.groupby([['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME'], pd.Grouper(key='T_H', freq='4H')])